In [ ]:
import requests
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from tqdm import tqdm
from joblib import Parallel, delayed
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

Перед парсингом мы поизучали сайт sokolov.ru и выяснили, что контент на страницах прогружается динамически - контент внизу страницы прогружается после того, как пользователь проскроллит страницу до него. Это значит, что не получится брать HTML-дерево страницы с помощью requests, нужно что-то придумывать.

Решением проблемы стало использование selenium, конкретнее - скрипт для скролла страницы до самого низа, после чего контент будет прогружен полностью.

Для начала напишем функции, собирающие ссылки на кольца из каталога (https://sokolov.ru/jewelry-catalog/rings/)

In [ ]:
def extract_links(ref_begin, page_count):
    links = []

    for i in tqdm(range(1, page_count + 1)):
        ref = f'{ref_begin}?page={i}'
        driver.get(ref)     # заходим на страницу с помощью selenium
        time.sleep(2)       # ждём прогрузку страницы
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    # скроллим страницу в самый низ
        time.sleep(2)       # ждём прогрузку страницы
        tree = BeautifulSoup(driver.page_source, 'html.parser')     # парсим дерево html с помощью bs4

        elems = tree.find('div', {'class' : 'ProductList_products-list__We3gK'}).find_all('a', {'data-qa-article':True})
        # по указанному тегу с атрибутом находим контейнер, который хранит все кольца, после чего находим все теги 'a' с атрибутом 'data-qa-article':True
        # - в них хранятся ссылки на кольца, сохраняем их в список elems

        #по каждому тегу 'a' из списка получаем ссылку, указанную в 'href', и добавляем её в список links
        for elem in elems:
            links.append('https://sokolov.ru/' + elem.get('href'))

    return links


# аналогичный вариант, но без скролла страницы в самый низ
def extract_links_2(ref_begin, page_count):
    links = []

    for i in tqdm(range(1, 147)):
        ref = f'{ref_begin}?page={i}'
        driver.get(ref)
        time.sleep(2)
        bs = BeautifulSoup(driver.page_source)
        rings = bs.find_all('a',attrs={'class':'ProductListItem_product-link__T8HTy'})
        links.extend([('https://sokolov.ru' + ring.get('href'), i) for ring in rings])
    driver.close
    return links

Применяем функцию, передаём в неё ссылку на каталог и число страниц - на момент парсинга 146, и собираем ссылки, после чего сохраняем их в отдельный текстовый файл `ring_links.txt`, чтобы не потерять.

In [ ]:
driver = webdriver.Chrome()
extracted_ring_links = extract_links_2('https://sokolov.ru/jewelry-catalog/rings/', 146)

ring_links = open('ring_links.txt', 'w')
for link in extracted_ring_links:
    ring_links.write(str(link[0]) + ' ' + str(link[1]))
    ring_links.write('\n')
ring_links.close()

100%|████████████████████████████████████████████████████████████████████████████████| 146/146 [08:44<00:00,  3.59s/it]


Далее напишем функцию, которая собирает данные по кольцам из ссылок.

In [ ]:
#пробовали в многопоточность - вышло не очень (не даёт значимый буст по времени), см. следующую функцию
def extract_data_threaded(ref):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(ref)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    try:
        tree = BeautifulSoup(driver.page_source, 'html.parser')

        chars_table = tree.find('div', {'class' : "Characteristics_sklv-characteristics__fdRmP"})
        chars_names = chars_table.find_all('div', {'class' : 'Characteristics_sklv-product-page-characteristics-row-name__wa7Gg'})
        chars_values = chars_table.find_all('div', {'class' : 'Characteristics_sklv-product-page-characteristics-row-vals__NwNL9'})

        rings_dict = {}

        for i in range(len(chars_names)):
            name = chars_names[i].span.get_text()
            value = chars_values[i].span.get_text()

            if name in rings_dict.keys():
                i = 2
                while (name + f' {i}') in rings_dict.keys():
                    i += 1
                name += f' {i}'

            rings_dict[name] = value

        rings_list.append(rings_dict)
        driver.close()
    except:
        problem_links.append(ref)
        driver.close()

def extract_data(ref, driver, p):
    driver.get(ref)     #заходим на кольцо через селениум
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    # скроллим вниз для прогрузки контента
    time.sleep(1)

    try:    #   ловим возможные ошибки - если что-то не вышло, добавляем ссылку на кольцо во внешний список problem_links (см. except)

        tree = BeautifulSoup(driver.page_source, 'html.parser')     # парсим дерево

        chars_table = tree.find('div', {'class' : "Characteristics_sklv-characteristics__fdRmP"})
        # по тегу div с атрибутом 'class' : "Characteristics_sklv-characteristics__fdRmP" находим контейнер, в котором находятся все таблицы с характеристиками кольца

        # внутри контейнера находим отдельно наименования характеристик и их значения по соответстующим тегам, и добавляем в два разных списка - отдельно имена, отдельно значения
        chars_names = chars_table.find_all('div', {'class' : 'Characteristics_sklv-product-page-characteristics-row-name__wa7Gg'})
        chars_values = chars_table.find_all('div', {'class' : 'Characteristics_sklv-product-page-characteristics-row-vals__NwNL9'})

        rings_dict = {}     # создаём словарь, описывающий одно кольцо - его будем добавлять во внешний список словарей ring_list

        for i in range(len(chars_names)):
            name = chars_names[i].span.get_text()       # для каждой характеристики забираем из списка её имя
            value = chars_values[i].span.get_text()     # и значение

            # генератор имени для признаков с повторяющимся названием (например, если уже есть характеристика "тип вставки", а попалась такая же, в словарь запихиваем её как "тип вставки 2")
            if name in rings_dict.keys():
                i = 2
                while (name + f' {i}') in rings_dict.keys():
                    i += 1
                name += f' {i}'

            rings_dict[name] = value    # добавляем в словарь пару 'имя' - 'значение'

        #Отдельно забираем цены со скидкой и без неё - они в другом контейнере

        rings_dict['Цена со скидкой'] = tree.find('div', {'class': 'ProductPrice_sklv-price__new__hdyUa'}).get('data-detail-price')

        # в контейнере, хранящем цену со скидкой, значение цены есть всегда (если по факту скидки нет, для кольца указано только одно значение цены).
        # А вот в контейнере, содержащем цену без скидки, значения может и не быть. Ловим такие ситуации через try - except:
        try:
            rings_dict['Цена без скидки'] = tree.find('div', {'class': 'ProductPrice_sklv-price__old__QlDQl'}).get('data-old-price')

        # если в try вылезла ошибка, значит кольцо не имеет скидки. Тогда в словарь добавим два одинаковых значения - цена со скидкой равна цене без скидки:
        except:
            rings_dict['Цена без скидки'] = rings_dict['Цена со скидкой']

        rings_dict['Номер страницы'] = p    # добавим в словарь также номер страницы каталога, на которой находилось кольцо. Номер страницы - p - передаётся как аргумент функции.

        rings_list.append(rings_dict)       # добавляем словарь кольца во внешний список

    except:
        problem_links.append(ref)

In [ ]:
# rings_list = []
# problem_links = []

# nj = -1
# Parallel(n_jobs=nj, prefer = 'threads')(
#     delayed(extract_data_threaded)(ref)
#     for ref in tqdm(extracted_ring_links))

Забираем из файла ссылки на кольца, а также номер страницы каталога, на котором они находятся. Список `extracted_ring_links` будет хранить пару `ссылка - номер страницы`

In [ ]:
extracted_ring_links = []

links_from_file = open('ring_links.txt', 'r')
for line in links_from_file:
    extracted_ring_links.append(line.strip().split())

In [ ]:
len(extracted_ring_links)

10476

Осталось запустить шайтан-машину.

In [ ]:
driver = webdriver.Chrome()
rings_list = []
problem_links = []

for i in tqdm(range(2414, len(extracted_ring_links))):
    extract_data(extracted_ring_links[i][0], driver, extracted_ring_links[i][1])

driver.close()

100%|████████████████████████████████████████████████████████████████████████████| 8062/8062 [5:40:47<00:00,  2.54s/it]


Так получилось, что на 2414 кольце упал интернет, поэтому парсили в два захода: сначала запускали цикл по всем 10476 кольцам, затем после падения на 2414 кольце перезапустили цикл, начиная с кольца 2414 (порядковый номер ссылки в списке `extracted_ring_links`). Первый заход сохранили в отдельный файл `output1.csv` (см далее)

In [ ]:
problem_links_file = open('problem_links.txt', 'w')
for link in problem_links:
    problem_links_file.write(link)
    problem_links_file.write('\n')
problem_links_file.close()

Ссылки с проблемами никак не обрабатывали, поскольку при открытии нескольких из них сайт выдавал ошибку загрузки. Примерное количество ссылок с такой ошибкой - 100. Было принято решение не рассматривать каждую ссылку по отдельности, так как проблемные ссылки составляют 1% от спаршенных данных.

Создаём `DataFrame` из полученного списка словарей и получаем на выходе таблицу, где одной строкой описывается одно кольцо.

In [ ]:
df = pd.DataFrame(rings_list)
df

,Артикул,Бренд,Коллекция,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,...,Цветность 9,Чистота 9,Цветность 10,Чистота 10,Тип вставки 14,Форма вставки 14,Количество 14,Цвет 14,Вес 14,Размеры вставки 14
0,93011090,SOKOLOV,RUSSE,Для женщин,2.54 г,Золочёное серебро,925,Золочение,Фианит,Сердце,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93011094,SOKOLOV,NaN,Для женщин,1 г,Золочёное серебро,925,Золочение,Фианит,Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,93011095,SOKOLOV,NaN,Для женщин,3.17 г,Золочёное серебро,925,"Золочение, Родирование, Эмаль",Эмаль,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,93011096,SOKOLOV,NaN,Для женщин,2.99 г,Золочёное серебро,925,"Золочение, Эмаль",Эмаль,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,93011100,SOKOLOV,NaN,Для женщин,1.23 г,Золочёное серебро,925,Золочение,Без вставок,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8013,94051161,SOKOLOV,SKLV,Для женщин,4.1 г,Родированное серебро,925,NaN,Фианит,Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8014,95010021,SOKOLOV,NaN,Для мужчин,12.34 г,Чернёное серебро,925,NaN,Фианит,Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8015,95010022,SOKOLOV,NaN,Для мужчин,12.37 г,Чернёное серебро,925,NaN,Фианит,Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8016,95010034,SOKOLOV,NaN,Для мужчин,9.12 г,Чернёное серебро,925,Чернение,Без вставок,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Сохраняем таблицу в файл (здесь написан код для второго захода на парсинг - после падения интернета, но для первого захода все операции были аналогичными)

In [ ]:
df.to_csv('output2.csv')

Читаем таблицы, полученные с обоих заходов на парсинг и объединяем их в один `DataFrame`, после чего сбрасываем индексы для удобства и сохраняем в итоговый файл `data.csv`

In [ ]:
df1 = pd.read_csv('output1.csv')
df2 = pd.read_csv('output2.csv')

finaldf = pd.concat([df1, df2])
finaldf.reset_index().drop(['index', 'Unnamed: 0'], axis=1).to_csv('data.csv')